In [7]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chat = ChatOpenAI(temperature=0.1)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

In [8]:
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

Chat model 은 대화에 최적화 되어 있는데 단지 질문을 받을 뿐 아니라, 대화를 할 수 있다는 뜻이다.

대화는, 그룹 오브 메세지스 즉, 여러 메세지의 묶음이라는 의미이다.

상대로서 대화의 맥락에 맞게 대답할 수 있다는 뜻이다. - Chat Model



만약 model 의 설정을 바꾸고 싶다면 

In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?"
    ),
]

SystemMessage 로 AI 에 일종의 기본 설정, 기본 값, 기본 context 설정을 함.

AIMessage 로 string 을 미리 만들어두고 일종의 가상 대화(마치 AI 가 답변한 것처럼)를 만들었음.

HumanMessage 로 사용자로서 질문을 함.

In [4]:
chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

predict 와 predict_messages 는 string(text) 과 messages 를 predict 하는 차이점이 있다.

---

# Prompt Template 에 대해 알아보자



prompt 는 중요하다. 왜? 우리가 LLM과 의사소통할 수 있는 유일한 방법이기 때문에.

prompt 성능이 좋으면 llm 의 성능도 좋을 것이다.

prompt template 의 종류는 두 가지로 볼 수 있다.
1. 일반 prompt template - template 를 string 을 이용해서 만든다.
2. chat prompt template - template 를 message 로 부터 만든다.

## normal prompt template

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thiland")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

## Chat Prompt Template

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# chat 을 상단에 위치
chat = ChatOpenAI(temperature=0.1)
# template 로 ChatPromptTemplate 의 from_messages 사용 - 리스트를 넘겨준다.
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are geography expert. And you only reply in {language}."),
        ("ai", "안녕, 내 이름은 {name}이야!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="Kim",
    country_a="Japan",
    country_b="China",
)

chat.predict_messages(prompt)

AIMessage(content='일본과 중국 사이의 거리는 대략 1500km입니다. 제 이름은 Kim입니다.')

# OutputParser 와 LCEL(표현언어)

lcel - 다양한 template 와 llm 호출, 그리고 서로 다른 응답(response)을 함께 사용하게 해준다.

outputparser 가 필요한 이유?

llm 의 응답을 변형해야 할 때가 있기 때문.

llm 은 항상 텍스트로 대답하기 때문에 그 응답을 무언가로 변형 시키고 싶은 경우가 생긴다.

In [13]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. everything you are asked will be answered with a comma seperate list of max {max_items} in lower case. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?",
)

chat.predict_messages(prompt)

AIMessage(content='mercury, venus, earth, mars, jupiter, saturn, uranus, neptune, pluto')

In [19]:
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

# Chain 
chain - 기본적으로 모든 요소를 합쳐주는 역할을 한다.

합쳐진 요소들은 하나의 chain 으로 실행된다. 하나하나 순서대로 result 를 반환할때까지.

chain 은 '|' 주변에 입력한 값들로 chain 을 형성한다. - LangChaing 의 핵심.

In [20]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

## Chain 은 Chain 끼리 결합할 수 있다.

### 2개의 체인 결합
chain_one = template | chat | CommaOuputParser()

chain_two = template_2 | chat | OutputParser()

### all = chain_one | chain_two | outputparser()

---


# LECL 에 대해 더 알아보자.